In [9]:
import psycopg2
import render_password as rpw

Exceptions 例外情況

[官方文件](https://www.psycopg.org/docs/module.html#exceptions)

In [10]:
#-----------寫入try, except，並利用psycopg2.Error傳出錯誤訊息-----------------
try:
    conn = psycopg2.connect(database=rpw.DATABASE, 
                            user=rpw.USER, 
                            password=rpw.PASSWORD, 
                            host=rpw.HOST, 
                            port="5432") #port是defult，固定填5432
except psycopg2.Error as e:
    print(e)

else:
    print(conn) #出現<connection object at 0x0000020C374A2AC0... --> 代表連線成功
    #conn.close() #連線後的conn一定要關閉

<connection object at 0x000001A0725439A0; dsn: 'user=rachelyah password=xxx dbname=maindb_eef7 host=dpg-cl5g50c72pts73elkvqg-a.singapore-postgres.render.com port=5432', closed: 0>


In [11]:
#----------------透過cursor直抓取資料庫的資料----------------------

sql = '''select * from 台北市youbike'''
with conn.cursor() as cursor:
    cursor.execute(sql)
    #cur.execute("SELECT * FROM my_data")
    records = cursor.fetchall()
#注意這裡上面的conn.colse()就不能執行，因為要維持練線狀態
print(records)

[(1, 'Youbike2.0_一壽橋', '2023-11-08 10:43:16', '文山區', '樟新街64號前方', 100, 100, 15), (2, 'Youbike2.0_一壽橋', '2023-11-08 10:43:18', '文山區', '樟新街64號前方', 100, 100, 15)]


In [13]:
#----------------寫入新的一筆資料-------------------------
sql = '''INSERT INTO 台北市youbike (站點名稱, 更新時間, 行政區, 地址, 總車輛數, 可借, 可還)
VALUES('Youbike2.0_一壽橋','2023-11-08 10:43:18', '文山區', '樟新街64號前方',100, 100, 15 )
'''

with conn.cursor() as cursor:
    cursor.execute(sql)
    conn.commit() #不會傳出值，請他執行的意思

#----------------抓取特定的的值------------------------
sql2 = '''
select * 
from 台北市youbike
where 站點名稱 = 'Youbike2.0_一壽橋'
'''

with conn.cursor() as cursor:
    cursor.execute(sql2)
    records = cursor.fetchall()

print(records)

SyntaxError: syntax error at end of input
LINE 4: 
        ^


In [20]:
#作業：寫入youbike1320筆資料
import csv

table_name = '台北市youbike'
csv_file = 'youbike.csv'
cur = conn.cursor()

with open(csv_file, 'r', newline='',encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
            station_name, update_time, district, address, total_bikes, available_bikes, available_parking = row
            sql = f"INSERT INTO {table_name} (站點名稱, 更新時間, 行政區, 地址, 總車輛數, 可借, 可還) " \
                f"VALUES ('{station_name}', '{update_time}', '{district}', " \
                f"'{address}', {total_bikes}, {available_bikes}, {available_parking});"
            cur.execute(sql)
conn.commit()
cur.close()

UniqueViolation: duplicate key value violates unique constraint "台北市youbike_站點名稱_更新時間_key"
DETAIL:  Key ("站點名稱", "更新時間")=(YouBike2.0_一壽橋, 2023-11-08 10:43:16) already exists.
